### Fitting Naive Bayes to the Training set

posterior = prior occurrances * liklihood / evidence
The "gaussian" and "naive" come from two assumptions present in this likelihood:
1. we assume each feature is uncorrelated from each other. That is, meannfreq is                  independent of sd or median etc.. This is obviously not true, and is a "naive"                assumption.
2. we assume have that the value of the features (e.g. meannfreq of women) are normally          (gaussian) distributed. This means that p(meannfreq∣female) is calculated by inputing          the required parameters into the probability density function of the normal                    distribution.


In [568]:
from sklearn.naive_bayes import GaussianNB
import pandas as pd 
import numpy as np  
import random
import math
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split  
from collections import defaultdict
from sklearn.model_selection import cross_val_score

In [569]:
%store -r train_x
%store -r test_x
%store -r train_y
%store -r test_y

%store -r train_x_two_features
%store -r test_x_two_features
%store -r train_y_two_features
%store -r test_y_two_features

%store -r transformed_train_x
%store -r transformed_test_x


In [570]:
def run_naiveBayes(train_x, test_x, train_y, test_y):
    classifier = GaussianNB()
    classifier.fit(train_x, train_y)
    print('In-Sample Accuracy:{:.4f}%'.format(classifier.score(train_x, train_y)*100))
    print('Out-of-Sample Accuracy:{:.4f}%'.format(classifier.score(test_x, test_y)*100))

In [571]:
print("with all features(Built-in Algorithm):")
run_naiveBayes(train_x, test_x, train_y, test_y)

with all features(Built-in Algorithm):
In-Sample Accuracy:89.5202%
Out-of-Sample Accuracy:90.0253%


In [572]:
#Second Approach: We wrote our own method to implement  https://machinelearningmastery.com/naive-bayes-classifier-scratch-python/
def calculateProbability(x, mean, variance_y):
    exp = math.exp(-(math.pow(x-mean, 2)/(2*math.pow(variance_y, 2))))
    result = (1/(math.sqrt(2*math.pi)*variance_y))*exp
    return result


In [573]:
def calculateClassProbabilities(data, inputVector):
    probabilities = {}
    for classValue, classdata in data.items():
        probabilities[classValue] = 1
        for i in range(len(classdata)):    
            mean, stdev = classdata[i]
            x = inputVector[i]
            probabilities[classValue] *= calculateProbability(x, mean, stdev)
    return probabilities

In [574]:
def predict(summaries, inputVector):
    probabilities = calculateClassProbabilities(summaries, inputVector)
    bestLabel, bestProb = None, -1
    for classValue, probability in probabilities.iteritems():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classValue
    return bestLabel

In [575]:
def getPredictions(summaries, testSet):
    predictions = []
    for i in range(len(testSet)):
        result = predict(summaries, testSet[i])
        predictions.append(result)
    return predictions

In [576]:
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [577]:
def separateByClass(dataset):
    separated = {}
    for i in range(len(dataset)):
        vector = dataset[i]
        if (vector[-1] not in separated):
            separated[vector[-1]] = []
        separated[vector[-1]].append(vector)
    return separated

def summarizeByClass(dataset):
    separated = separateByClass(dataset)
    summaries = {}
    for classValue, instances in separated.iteritems():
        summaries[classValue] = summarize(instances)
    return summaries


In [578]:
# # For both the classes 0 and 1, and for each of the 20 features, we have thus saved the mean and the standard in summary. 
# # For example, for the class 0 (female), and feature 1 (ie, sd), mean and the standard for this attribute are 0.060910268026077803 and 0.00034668419803097784

# def getSummary(data,n_cls):
#     data_means = data.groupby('label').mean()
#     data_variance = data.groupby('label').var()
    
#     cnames = np.array(data.columns.tolist())[:-1] 
#     classes = np.unique(labels)
#     classnames = np.array(['female','male'])
#     summary = {}

#     for cls in classes:
#         #initializing the dictionary
#         summary[cls] = defaultdict(list)
#     for cls in classes:
#         for j in range(0, n_cls):
#             print(classnames[cls])
#             summary[cls][j] += list(np.array([data_means[cnames[j]][data_means.index == classnames[cls]].values[0],
#                                           data_variance[cnames[j]][data_variance.index == classnames[cls]].values[0]]))
            
#     print summary   
#     return summary

In [579]:
# summary = getSummary(train_df,train_x.shape[1])

In [580]:
# # Group the data by gender and calculate the means of each feature
# data_means = train_x_df.groupby('label').mean()
# # View the values
# data_means

In [581]:
# # Group the data by gender and calculate the variance of each feature
# data_variance = train_x_df.groupby('label').var()
# # View the values
# data_variance

In [582]:
# # Create a function to calculate the probability density of each of the terms of the likelihood 
# # We use mean and standard deviation of input values (x) for each class to summarize the distribution
# # Probabilities of new x values are calculated using the Gaussian Probability Density Function (PDF).

# def p_x_given_y(x, mean_y, variance_y):
#     # Input the arguments into a probability density function
#     p = 1/(np.sqrt(2*np.pi*variance_y)) * np.exp((-(x-mean_y)**2)/(2*variance_y))
#     return p

# # We calculate the probability that the value of the random variable will be between x - 0.01 and x + 0.01


In [583]:
# # Test
# p_x_given_y(2, 2, 4)

In [584]:
# def calculateClassProbabilities(summaries, inputVector):
#     # p(x|y) (e.g. p(meannfreq∣female))
#     probabilities = {}
#     for classValue, classSummaries in summaries.iteritems():
#         if (classValue == 0):
#             probabilities[classValue] = P_female
        
#         else:
#             probabilities[classValue] = P_male
#         for i in range(len(classSummaries)):
#             mean, var = classSummaries[i]
#             x = inputVector[i]
#             probabilities[classValue] *= p_x_given_y(x, mean, var)
#     return probabilities

In [585]:
# calculateClassProbabilities(summary, test_x[10])
# # PDF can be greater than 1, but it can't be greater than 1 for a large interval
# # because of the non-negativity and integral constraints.

In [586]:
# # Make a Prediction
# import operator
# def predict(summaries, inputVector):
#     probabilities = calculateClassProbabilities(summaries, inputVector)
#     #print probabilities
#     probabilities = sorted(probabilities.iteritems(), key=operator.itemgetter(1), reverse=True)
#     return probabilities[0][0]

In [587]:
# print(test_x[10])
# print('Our prediction for test_x[10] is %s'  % (predict(summary, test_x[10])))

In [588]:
# from sklearn.metrics import classification_report

# def GaussianNB_Classifier(test_x,test_y):
#     results = []
#     for i in range(len(test_x)):
#         test_single = test_x[i]
#         res = predict(summary, test_single)
#         results.append(res)
#     # We evaluate the accuracy of the model by calculating a ratio of the total correct predictions out of all predictions made (the classification accuracy)
#     correct = 0
#     wrong = 0
#     for j in range(len(test_y)):
#         if(results[j] == test_y[j]):
#             correct += 1
#         else:
#             wrong += 1
 
#     print('Correct rate is %s'  % (float(correct)/(correct+wrong)))
#     print(classification_report(test_y,results))



In [589]:
# print("Use classification reports to evaluate how well Gaussian Naive Bayes performed with all features \n")
# print("in-sample accuracy in Gaussian Naive Bayes:")
# GaussianNB_Classifier(train_x,train_y)

In [590]:
# print("out-of-sample accuracy in Gaussian Naive Bayes:")
# GaussianNB_Classifier(test_x,test_y)

In [591]:
# # We are wondering if there is a big difference in the results 
# # when used Sklearn package vs. our own algorithm
# from sklearn.naive_bayes import GaussianNB
# from sklearn.metrics import accuracy_score

# nb = GaussianNB()
# nb.fit(train_x, train_y)
# prediction = nb.predict(test_x)

# print('NB result: ', accuracy_score(test_y, prediction))

In [592]:
# # Is it enough of these two features to make predictions? 
# ## Random Forest with 2 features ('meanfun', 'IQR')

# voice_new_data = voice_data[['IQR','meanfun','label']]
# train_new_x = train_x[:,[5,12]]
# test_new_x = test_x[:,[5,12]]
# summary = getSummary(voice_new_data, train_new_x.shape[1])

In [593]:
# print("Use classification reports to evaluate how well Gaussian Naive Bayes performed with only 2 features \n")
# print("In-sample accuracy in Gaussian Naive Bayes:")
# GaussianNB_Classifier(train_new_x,train_y)

In [594]:
# print("Out-of-sample accuracy in Gaussian Naive Bayes:")
# GaussianNB_Classifier(test_new_x,test_y)